In [1]:
import requests
import xml.etree.ElementTree as ET  # for parsing XML

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
from time import sleep
from tqdm import tqdm
mongodb_uri = os.environ.get("MONGODB_URI")

In [3]:
base_semester_url = "https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall.xml"

subjects_list = []

subjects_root = ET.fromstring(requests.get(base_semester_url).content)
subjects_xml = subjects_root.find("subjects").findall("subject")

In [4]:
courses_list = []

for subject in tqdm(subjects_xml) : 
    course_root = ET.fromstring(requests.get(subject.attrib["href"]).content)
    course_xml = course_root.find("courses").findall("course")
    for course in course_xml:
        courses_list.append({
            "name" : course.text,
            "number" : course.attrib["id"],
            "code" : subject.attrib["id"],
            "api_link" : course.attrib["href"],
            "year" : 2024,
            "semester" : "fall"
        })

  0%|          | 0/185 [00:00<?, ?it/s]

100%|██████████| 185/185 [00:30<00:00,  6.07it/s]


In [5]:
print(courses_list)

[{'name': 'Intro Asian American Studies', 'number': '100', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall/AAS/100.xml', 'year': 2024, 'semester': 'fall'}, {'name': 'U.S. Race and Empire', 'number': '200', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall/AAS/200.xml', 'year': 2024, 'semester': 'fall'}, {'name': 'US Racial & Ethnic Politics', 'number': '201', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall/AAS/201.xml', 'year': 2024, 'semester': 'fall'}, {'name': 'US Citizenship Comparatively', 'number': '215', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall/AAS/215.xml', 'year': 2024, 'semester': 'fall'}, {'name': 'Intro Asian American Theatre', 'number': '260', 'code': 'AAS', 'api_link': 'https://courses.illinois.edu/cisapp/explorer/schedule/2024/fall/AAS/260.xml', 'year': 2024, 'semester': 'fall'}, {'name': 'Con

In [6]:
import pymongo
client = pymongo.MongoClient(mongodb_uri)
db = client[os.environ.get("MONGODB_DB")]

In [7]:
classes = db["classes"]
classes.insert_many(courses_list)